In [2]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
import scipy.stats as ss
from scipy import optimize
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import pandas as pd
from scipy.integrate import quad
from itertools import product

**Задание 1**


На лекции мы уже проводили те же самые вычисления и получали:

$$ n \geq \frac{11.2^2\sigma^2}{\delta_i^2} = \frac{16 \cdot 2.8^2 \sigma^2}{\delta_i^2}$$

Правда, мы грубо оценили 
$ss.norm().ppf(0.975) - ss.norm().ppf(0.2) \approx 2.8$

Можно посчитать точнее:

In [19]:
# ss.norm().ppf(0.975) - ss.norm().ppf(0.2) = 2.8015852181129683

print("n >=", 16 * (ss.norm().ppf(0.975) - ss.norm().ppf(0.2)) ** 2  * 10, '=> n >= ', np.ceil( 16 * (ss.norm().ppf(0.975) - ss.norm().ppf(0.2)) ** 2  * 10))

n >= 1255.8207574958542 => n >=  1256.0


**Задание 2**

In [29]:
data = pd.read_csv('test.csv')

G = {}
for i in ['control', 'test']:
  for j in ['yes', 'no']:
    G[(i,j)] = np.array(data[(data['group'] == i) & (data['smoker'] == j)]['effect'])

x_a = G['control', 'yes'] - G['control', 'no']
x_b = G['test', 'yes'] - G['test', 'no']

alpha = 0.05
z = ss.norm().ppf(1 - alpha/2)
n = len(data)
d = 4 * np.sqrt(10 / n)
(x_b.mean() - x_a.mean()) + d * np.array([-z, z])

array([0.19320119, 1.30192425])

**Задание 3**

In [24]:
table = {(0, 0, 0): 0,
 (0, 0, 1): 0,
 (0, 0, 2): 0,
 (0, 0, 3): 0,
 (0, 0, 5): 0,
 (0, 0, 6): 2,
 (0, 0, 7): 0,
 (0, 1, 1): 0,
 (0, 1, 2): 0,
 (0, 1, 3): 0,
 (0, 1, 5): 0,
 (0, 1, 6): 2,
 (0, 1, 7): 0,
 (0, 2, 2): 0,
 (0, 2, 3): 0,
 (0, 2, 5): 0,
 (0, 2, 6): 2,
 (0, 2, 7): 0,
 (0, 3, 3): 0,
 (0, 3, 5): 0,
 (0, 3, 6): 2,
 (0, 3, 7): 0,
 (0, 5, 5): 0,
 (0, 5, 6): 10,
 (0, 5, 7): 0,
 (0, 6, 6): 5,
 (0, 6, 7): 2,
 (0, 7, 7): 0,
 (1, 1, 1): 10,
 (1, 1, 2): 5,
 (1, 1, 3): 5,
 (1, 1, 5): 20,
 (1, 1, 6): 2,
 (1, 1, 7): 0,
 (1, 2, 2): 5,
 (1, 2, 3): 5,
 (1, 2, 5): 10,
 (1, 2, 6): 2,
 (1, 2, 7): 0,
 (1, 3, 3): 5,
 (1, 3, 5): 10,
 (1, 3, 6): 2,
 (1, 3, 7): 0,
 (1, 5, 5): 40,
 (1, 5, 6): 10,
 (1, 5, 7): 0,
 (1, 6, 6): 5,
 (1, 6, 7): 2,
 (1, 7, 7): 0,
 (2, 2, 2): 25,
 (2, 2, 3): 5,
 (2, 2, 5): 50,
 (2, 2, 6): 2,
 (2, 2, 7): 0,
 (2, 3, 3): 5,
 (2, 3, 5): 10,
 (2, 3, 6): 2,
 (2, 3, 7): 0,
 (2, 5, 5): 100,
 (2, 5, 6): 10,
 (2, 5, 7): 0,
 (2, 6, 6): 5,
 (2, 6, 7): 2,
 (2, 7, 7): 0,
 (3, 3, 3): 40,
 (3, 3, 5): 80,
 (3, 3, 6): 2,
 (3, 3, 7): 0,
 (3, 5, 5): 160,
 (3, 5, 6): 10,
 (3, 5, 7): 0,
 (3, 6, 6): 5,
 (3, 6, 7): 2,
 (3, 7, 7): 0,
 (5, 5, 5): 1000,
 (5, 5, 6): 40,
 (5, 5, 7): 320,
 (5, 6, 6): 20,
 (5, 6, 7): 10,
 (5, 7, 7): 160,
 (6, 6, 6): 10,
 (6, 6, 7): 5,
 (6, 7, 7): 2,
 (7, 7, 7): 80}

with open('slot_138 .txt', 'r') as file:
    data = []
    for row in file.readlines():
        data += list(map(int, row.strip().split()))

vals, counts = np.unique(data, return_counts=True)
n = counts.sum()


def likely_hood(theta):
    return - (np.log(theta) * counts).sum()

sum_equal_one = LinearConstraint(np.ones(7), 1, 1)

M = np.array([table[tuple(sorted(x))] for x in product([0, 1, 2, 3, 5, 6, 7], repeat=3)]).reshape((7, 7, 7))
def make_mean(p):
    P = p[:, None, None] * p[None, :, None] * p[None, None, :]
    return (M * P).sum()

mean_is_fixed = NonlinearConstraint(make_mean, 0.92, 0.92)

bounds=[(0.000001, 0.999999999) for _ in range(7)]
theta0 = np.ones(7)/7

res = minimize(likely_hood, x0=theta0, bounds=bounds, constraints=[sum_equal_one, mean_is_fixed])
theta_star = res.x

ss.chisquare(counts, n * theta_star, ddof=5).pvalue

0.2844672940009947

**Задача 4**

In [25]:
data = pd.read_csv('view_proxy_randomization.csv')
vals, counts = np.unique(data['category'], return_counts=True)
n = counts.sum()
ss.chisquare(counts, n * np.ones(11) / 11, ddof=0).pvalue

0.36595976450243883

**Задача 5**

In [26]:
ss.ttest_ind(data[data['category'] == 5]['VIEW'], data[data['category'] == 10]['VIEW'], equal_var=False).statistic

4.199979869026695